In [27]:
import os
import numpy as np
import pandas as pd

from torch.utils.data import random_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset

In [28]:
data = []
col = ["Date","HomeTeam","AwayTeam","FTR","B365H","B365D",
      "B365A","BWH","BWD","BWA","IWH","IWD","IWA","PSH","PSD","PSA","WHH","WHD","WHA",
      "VCH","VCD","VCA"]
for i in range(21,11,-1):
    temp = pd.read_csv(f"./data/EPL/{i}_{i+1}.csv")
    temp = temp[col]
    temp["Season"] = f"{i}/{i+1}"
    print(f"{i}/{i+1} season ok")
    data.append(temp)

21/22 season ok
20/21 season ok
19/20 season ok
18/19 season ok
17/18 season ok
16/17 season ok
15/16 season ok
14/15 season ok
13/14 season ok
12/13 season ok


In [29]:
print("data length : " + str(len(data)))
for i in range(len(data)) :
    print(f"data[{i}].shape = {data[i].shape}")

data length : 10
data[0].shape = (110, 23)
data[1].shape = (380, 23)
data[2].shape = (380, 23)
data[3].shape = (380, 23)
data[4].shape = (380, 23)
data[5].shape = (380, 23)
data[6].shape = (380, 23)
data[7].shape = (381, 23)
data[8].shape = (380, 23)
data[9].shape = (380, 23)


In [32]:
total = pd.concat(data)
print(f"total shape = {total.shape}")

total shape = (3531, 23)


In [33]:
total.head()

,Date,HomeTeam,AwayTeam,FTR,B365H,B365D,B365A,BWH,BWD,BWA,...,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,Season
0,13/08/2021,Brentford,Arsenal,H,4.00,3.40,1.95,4.00,3.50,1.95,...,4.05,3.46,2.05,4.00,3.40,1.90,4.10,3.40,2.00,21/22
1,14/08/2021,Man United,Leeds,H,1.53,4.50,5.75,1.53,4.50,5.75,...,1.56,4.57,5.96,1.52,4.33,5.80,1.55,4.40,6.00,21/22
2,14/08/2021,Burnley,Brighton,A,3.10,3.10,2.45,3.20,3.10,2.40,...,3.30,3.12,2.51,3.20,3.00,2.45,3.13,3.10,2.45,21/22
3,14/08/2021,Chelsea,Crystal Palace,H,1.25,5.75,13.00,1.28,5.75,10.50,...,1.26,6.24,12.74,1.25,5.50,13.00,1.25,5.75,13.00,21/22
4,14/08/2021,Everton,Southampton,H,1.90,3.50,4.00,1.95,3.50,3.90,...,2.01,3.56,4.10,1.95,3.40,4.00,1.95,3.40,4.10,21/22


In [67]:
team_name = total["HomeTeam"].unique()


object


In [68]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [72]:
le.fit(team_name)
team_label = le.transform(team_name)
print(f"team name : {team_name}")
print(f"team_label : {team_label}")

team name : ['Brentford' 'Man United' 'Burnley' 'Chelsea' 'Everton' 'Leicester'
 'Watford' 'Norwich' 'Newcastle' 'Tottenham' 'Liverpool' 'Aston Villa'
 'Crystal Palace' 'Leeds' 'Man City' 'Brighton' 'Southampton' 'Wolves'
 'Arsenal' 'West Ham' 'Fulham' 'West Brom' 'Sheffield United'
 'Bournemouth' 'Huddersfield' 'Cardiff' 'Stoke' 'Swansea' 'Hull'
 'Middlesbrough' 'Sunderland' 'QPR' nan 'Reading' 'Wigan']
team_label : [ 3 17  5  7  9 14 29 20 19 28 15  1  8 13 16  4 24 33  0 31 10 30 23  2
 11  6 25 27 12 18 26 21 34 22 32]


In [73]:
dic = {}
for i in range(len(team_label)) :
    dic[team_name[i]] = team_label[i]

In [75]:
dic

{'Brentford': 3,
 'Man United': 17,
 'Burnley': 5,
 'Chelsea': 7,
 'Everton': 9,
 'Leicester': 14,
 'Watford': 29,
 'Norwich': 20,
 'Newcastle': 19,
 'Tottenham': 28,
 'Liverpool': 15,
 'Aston Villa': 1,
 'Crystal Palace': 8,
 'Leeds': 13,
 'Man City': 16,
 'Brighton': 4,
 'Southampton': 24,
 'Wolves': 33,
 'Arsenal': 0,
 'West Ham': 31,
 'Fulham': 10,
 'West Brom': 30,
 'Sheffield United': 23,
 'Bournemouth': 2,
 'Huddersfield': 11,
 'Cardiff': 6,
 'Stoke': 25,
 'Swansea': 27,
 'Hull': 12,
 'Middlesbrough': 18,
 'Sunderland': 26,
 'QPR': 21,
 nan: 34,
 'Reading': 22,
 'Wigan': 32}